In [1]:
import findspark
findspark.init("/home/chenekv/spark-3.5.4-bin-hadoop3-scala2.13")

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, row_number
from pyspark.sql.window import Window

spark = SparkSession.builder \
    .appName("CityAnalysisTasks") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

balance_file = "/home/chenekv/code/jupyterlab/Purchase Redemption Data/user_balance_table.csv"
balance_df = spark.read.csv(balance_file, header=True, inferSchema=True)

profile_file = "/home/chenekv/code/jupyterlab/Purchase Redemption Data/user_profile_table.csv"
profile_df = spark.read.csv(profile_file, header=True, inferSchema=True)

print("Balance table columns:", balance_df.columns)
print("Profile table columns:", profile_df.columns)

24/12/21 17:46:31 WARN Utils: Your hostname, chenekv-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
24/12/21 17:46:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/21 17:46:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/21 17:46:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/21 17:46:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
                                                                                

Balance table columns: ['user_id', 'report_date', 'tBalance', 'yBalance', 'total_purchase_amt', 'direct_purchase_amt', 'purchase_bal_amt', 'purchase_bank_amt', 'total_redeem_amt', 'consume_amt', 'transfer_amt', 'tftobal_amt', 'tftocard_amt', 'share_amt', 'category1', 'category2', 'category3', 'category4']
Profile table columns: ['user_id', 'sex', 'city', 'constellation']


In [2]:
combined_df = balance_df.join(profile_df, on="user_id", how="inner")

In [3]:
march_1_data = combined_df.filter(col("report_date") == 20140301)

city_avg_balance = march_1_data.groupBy("city").agg(
    avg("tBalance").alias("average_balance")
)

sorted_city_avg_balance = city_avg_balance.orderBy(col("average_balance").desc())

print("任务1：每个城市在2014年3月1日的用户平均余额")
sorted_city_avg_balance.show()

output_path_task1 = "/home/chenekv/code/jupyterlab/city_average_balance_output"
sorted_city_avg_balance.write.csv(output_path_task1, header=True)

任务1：每个城市在2014年3月1日的用户平均余额
+-------+------------------+
|   city|   average_balance|
+-------+------------------+
|6281949| 2795923.837298216|
|6301949|2650775.0664451825|
|6081949|2643912.7566638007|
|6481949|2087617.2136986302|
|6411949|1929838.5617977527|
|6412149| 1896363.471625767|
|6581949|1526555.5551020408|
+-------+------------------+



In [4]:
august_data = combined_df.filter((col("report_date") >= 20140801) & (col("report_date") <= 20140831))

city_user_flow = august_data.groupBy("city", "user_id").agg(
    sum(col("total_purchase_amt") + col("total_redeem_amt")).alias("total_flow")
)

windowSpec = Window.partitionBy("city").orderBy(col("total_flow").desc())

top3_users_per_city = city_user_flow.withColumn("rank", row_number().over(windowSpec)) \
    .filter(col("rank") <= 3) \
    .select("city", "user_id", "total_flow")

print("任务2：每个城市总流量前3高的用户")
top3_users_per_city.show()

output_path_task2 = "/home/chenekv/code/jupyterlab/city_top_users_output"
top3_users_per_city.write.csv(output_path_task2, header=True)

任务2：每个城市总流量前3高的用户


+-------+-------+----------+
|   city|user_id|total_flow|
+-------+-------+----------+
|6081949|  27235| 108475680|
|6081949|  27746|  76065458|
|6081949|  18945|  55304049|
|6281949|  15118| 149311909|
|6281949|  11397| 124293438|
|6281949|  25814| 104428054|
|6301949|   2429| 109171121|
|6301949|  26825|  95374030|
|6301949|  10932|  74016744|
|6411949|    662|  75162566|
|6411949|  21030|  49933641|
|6411949|  16769|  49383506|
|6412149|  22585| 200516731|
|6412149|  14472| 138262790|
|6412149|  25147|  70594902|
|6481949|  12026|  51161825|
|6481949|    670|  49626204|
|6481949|  14877|  34488733|
|6581949|   9494|  38854436|
|6581949|  26876|  23449539|
+-------+-------+----------+
only showing top 20 rows

